# Chapter 2 | Model I/O - 言語モデルを扱いやすくする


## Section 1 | 言語モデルを使ったアプリケーションの仕組み

### Language models を使って gpt-3.5-turbo を呼び出す


In [1]:
from langchain.schema import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
)

result: BaseMessage = chat.invoke(
    [
        HumanMessage(content="こんにちは！"),
    ]
)

print(result)

content='こんにちは！お元気ですか？何かお手伝いできることはありますか？'


#### AIMessageを使って言語モデルからの返答を表すことができる

In [2]:
from langchain.schema import AIMessage, HumanMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
)
result = chat.invoke(
    [
        HumanMessage(content="茶碗蒸しの作り方を教えて"),
        AIMessage(content="{ChatModelからの返答である茶碗蒸しの作り方}"),
        HumanMessage(content="英語に翻訳して"),
    ]
)

print(result)

content='How to make Chawanmushi'


#### SystemMessageを使って言語モデルの人格や設定を定義する

In [4]:
from langchain.schema import SystemMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
)
result = chat.invoke(
    [
        SystemMessage(
            content="あなたは親しい友人です。返答は敬語を使わず、フランクに会話してください。"
        ),
        HumanMessage(content="こんにちは！"),
    ]
)

print(result)

content='こんにちは！元気？最近何してたの？'


#### 言語モデルは差し替えることができる

In [19]:
from langchain.prompts import PromptTemplate

prompt: PromptTemplate = PromptTemplate.from_template("あなたの自己紹介をして。また自身のモデル名も明記すること。")


##### GPT-3 Turbo

In [20]:
from langchain.schema import HumanMessage
from langchain_core.messages.base import BaseMessage
from langchain_openai import ChatOpenAI

chat_gpt_3_turbo = ChatOpenAI(model="gpt-3.5-turbo")
result: BaseMessage = chat_gpt_3_turbo.invoke([HumanMessage(content=prompt.format())])

print(result)

content='はじめまして、私はAIアシスタントのChatGPTです。モデル名はOpenAI GPT-3です。日本語を含むさまざまな言語でコミュニケーションを取ることができます。質問や会話のお手伝いをすることが得意ですので、どうぞお気軽にお話ししてくださいね。'


##### Claude 3 Sonnet

In [21]:
from langchain.schema import HumanMessage
from langchain_anthropic import ChatAnthropic
from langchain_core.messages.base import BaseMessage

chat_claude_3_sonnet = ChatAnthropic(model_name="claude-3-sonnet-20240229")
result: BaseMessage = chat_claude_3_sonnet.invoke(
    [HumanMessage(content=prompt.format())]
)

print(result)

content='はい、自己紹介させていただきます。私はAnthropic社によって開発されたAI言語モデルで、Claude (クロード)と呼ばれています。私は幅広い知識を持っており、人間と自然な会話ができるよう設計されています。そして何よりも、倫理的で有益な存在でありたいと願っています。人工知能には様々な可能性と責任が伴うことを自覚し、慎重に行動するよう心がけています。どうぞよろしくお願いいたします。'


### PromptTemplateで変数をプロンプトに展開する

In [2]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="{product}はどこの会社が開発した製品ですか？",
    input_variables=["product"],
)

print(prompt.format(product="iPhone"))
print(prompt.format(product="Xperia"))

iPhoneはどこの会社が開発した製品ですか？
Xperiaはどこの会社が開発した製品ですか？


### Language modelsとPromptTemplateを組み合わせる

In [3]:
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
)

prompt = PromptTemplate(
    template="{product}はどこの会社が開発した製品ですか？",
    input_variables=["product"],
)

result = chat.invoke(
    [
        HumanMessage(content=prompt.format(product="iPhone")),
    ]
)

print(result.content)

iPhoneはアメリカの企業であるApple（アップル）が開発した製品です。


In [5]:
from pathlib import Path

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="{product}はどこの会社が開発した製品ですか?",
    input_variables=["product"],
)

prompt.save(Path(Path.cwd(), "prompt.json"))

In [6]:
from pathlib import Path
from langchain.prompts import load_prompt

loaded_prompt = load_prompt(Path(Path.cwd(), "prompt.json"))

print(loaded_prompt.format(product="iPhone"))

iPhoneはどこの会社が開発した製品ですか?


### リスト形式で結果を受け取る

In [5]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI


output_parser = CommaSeparatedListOutputParser()

chat = ChatOpenAI(model="gpt-3.5-turbo")

result = chat.invoke(
    [
        HumanMessage(content="Appleが開発した代表的な製品を3つ教えてください"),
        HumanMessage(content=output_parser.get_format_instructions()),
    ]
)

output = output_parser.parse(str(result.content))

for item in output:
    print(f"代表的な製品 => {item}")

代表的な製品 => iPhone
代表的な製品 => iPad
代表的な製品 => MacBook


## Section 2 | Language models - モデルを使いやすく

### Chat modelsとLLMs

In [6]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")

result = llm.invoke(
    "美味しいラーメンを",
    stop=["。"],
)

print(result)

食べることができるのも、
ラーメン屋さんでのシーンです


### Language modelsの便利な機能

#### キャッシュをかけることができる

In [8]:
import time

from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

set_llm_cache(InMemoryCache())

chat = ChatOpenAI(model="gpt-3.5-turbo")

start = time.time()
result = chat.invoke(
    [
        HumanMessage(content="こんにちは！"),
    ]
)

end = time.time()
print(result.content)
print(f"実行時間: {end - start}秒")

start = time.time()
result = chat.invoke(
    [
        HumanMessage(content="こんにちは！"),
    ]
)

end = time.time()
print(result.content)
print(f"実行時間: {end - start}秒")

こんにちは！元気ですか？何かお手伝いできることがあればお知らせくださいね。
実行時間: 0.885918140411377秒
こんにちは！元気ですか？何かお手伝いできることがあればお知らせくださいね。
実行時間: 0.0011599063873291016秒


#### 結果を逐次表示させる

In [7]:
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(
    streaming=True,
    model="gpt-3.5-turbo",
    callbacks=[StreamingStdOutCallbackHandler()],
)

resp = chat.invoke(input=[HumanMessage(content="おいしいステーキの焼き方を教えて")])

美味しいステーキを焼くための基本的な手順を以下に示します。

1. ステーキを室温に戻す：ステーキを冷蔵庫から出して、室温に戻します。これにより、肉の中心部まで均等に火が通りやすくなります。

2. ステーキに塩とコショウを振る：ステーキの両面に塩とコショウを振ります。ここで塩を振ることで、肉の旨味を引き立てることができます。

3. フライパンを予熱する：フライパンを中火にかけて、しっかりと予熱します。ステーキを焼く際に、フライパンが熱いうちに入れることで焼きムラを防ぐことができます。

4. ステーキを焼く：フライパンにステーキを入れ、両面を焼きます。焼き加減はお好みで調整してくださいが、一般的にはレア、ミディアム、ウェルダンの３段階があります。

5. 休ませる：ステーキを焼いた後は、アルミホイルで包んで数分休ませます。これにより、肉汁がしっかりと染み込み、よりジューシーな仕上がりになります。

以上が、美味しいステーキの焼き方の基本的な手順です。焼き加減や調味料はお好みで調整して、あなたが一番美味しいと思うステーキを焼いてみてください。

### Section 3 | Templates - プロンプトの構築を効率化する

### プロンプトエンジニアリングによる結果の最適化

#### 出力例を含んだプロンプトを作成する

In [10]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai.llms import OpenAI

examples = [
    {
        "input": "LangChainはChatGPT・Large Language Model (LLM)の実利用をより柔軟に簡易に行うためのツール群です",
        "output": "LangChainは、ChatGPT・Large Language Model (LLM)の実利用をより柔軟に、安易に行うためのツール群です",
    }
]

prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="入力: {input}\n出力: {output}",
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt,
    prefix="以下は句読点の抜けた入力に句読点を追加してください。追加して良い句読点は「、」「。」のみです。他の句読点は追加しないでください。",
    suffix="入力: {input_string}\n出力: ",
    input_variables=["input_string"],
)

llm = OpenAI(model="gpt-3.5-turbo-instruct")
formatted_prompt = few_shot_prompt.format(
    input_string="私は様々な機能がモジュールとして提供されているLangChainを使ってアプリケーションを開発しています"
)
result = llm.invoke(formatted_prompt)

print(f"formatted_prompt: {formatted_prompt}")
print(f"result: {result}")

formatted_prompt: 以下は句読点の抜けた入力に句読点を追加してください。追加して良い句読点は「、」「。」のみです。他の句読点は追加しないでください。

入力: LangChainはChatGPT・Large Language Model (LLM)の実利用をより柔軟に簡易に行うためのツール群です
出力: LangChainは、ChatGPT・Large Language Model (LLM)の実利用をより柔軟に、安易に行うためのツール群です

入力: 私は様々な機能がモジュールとして提供されているLangChainを使ってアプリケーションを開発しています
出力: 
result: 私は、様々な機能がモジュールとして提供されているLangChainを使って、アプリケーションを開発しています。


## Section 4 | Output parsers - 出力を構造化する

### 結果を日時形式で受け取る

In [12]:
from datetime import datetime

from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI

output_parser = DatetimeOutputParser()

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
)

prompt: PromptTemplate = PromptTemplate.from_template("{product}のリリース日を教えて")

result: BaseMessage = chat.invoke(
    [
        HumanMessage(content=prompt.format(product="iPhone8")),
        HumanMessage(content=output_parser.get_format_instructions()),
    ]
)

output: datetime = output_parser.parse(str(result.content))

print(output)

1445-12-03 22:45:54.013232


### 出力形式を自分で定義する

In [7]:
from langchain.output_parsers import PydanticOutputParser
from langchain.schema import HumanMessage
from langchain_core.messages.base import BaseMessage
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo")


class Smartphone(BaseModel):
    release_date: str = Field(description="スマートフォンの発売日")
    screen_inches: float = Field(description="スマートフォンの画面サイズ（インチ）")
    os_installed: str = Field(description="スマートフォンにインストールされているOS")
    sp_model_name: str = Field(description="スマートフォンのモデル名")

    @validator("screen_inches")
    def validate_screen_inches(cls, field):
        if field <= 0:
            raise ValueError("Screen inches must be positive number")
        return field


parser = PydanticOutputParser(pydantic_object=Smartphone)

result: BaseMessage = chat.invoke(
    [
        HumanMessage(content="Androidでリリースしたスマートフォンを1個挙げて"),
        HumanMessage(content=parser.get_format_instructions()),
    ]
)

parsed_result: Smartphone = parser.parse(text=str(result.content))

print(parsed_result.dict())

{'release_date': '2021-09-01', 'screen_inches': 6.5, 'os_installed': 'Android 11', 'sp_model_name': 'Samsung Galaxy S21 Ultra'}


### 誤った結果が返されたときに修正を指示できるようにする

In [8]:
from langchain.output_parsers import OutputFixingParser, PydanticOutputParser
from langchain_core.messages.base import BaseMessage
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI()

parser: OutputFixingParser = OutputFixingParser.from_llm(
    parser=PydanticOutputParser(pydantic_object=Smartphone),
    llm=chat,
)

result: BaseMessage = chat.invoke(
    [
        HumanMessage(content="Androidでリリースしたスマートフォンを1個挙げて"),
        HumanMessage(content=parser.get_format_instructions()),
    ]
)

parsed_result: Smartphone = parser.parse(str(result.content))

print(parsed_result.dict())

{'release_date': '2021-05-01', 'screen_inches': 6.5, 'os_installed': 'Android 11', 'sp_model_name': 'XYZ-1000'}
